In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [30]:
%set_env OPENAI_API_KEY=
%set_env TAVILY_API_KEY=

env: OPENAI_API_KEY=
env: TAVILY_API_KEY=


In [3]:
tool = TavilySearchResults()

In [5]:
class AgentState(TypedDict):
    messages:Annotated[list[AnyMessage],operator.add]

In [6]:
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver.from_conn_string(":memory:")

In [19]:
class Agent:
    def __init__(self,model,tools,checkpointer,system=""):
        self.system=system
        graph = StateGraph(AgentState)
        graph.add_node("llm",self.call_openai)
        graph.add_node("action",self.take_action)
        graph.add_conditional_edges("llm",self.exists_action,{True:"action",False:END})
        graph.add_edge("action","llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name:t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self,state:AgentState):
        messages=state['messages']
        if self.system:
            messages=[SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages":[message]}
    def exists_action(self,state:AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [20]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI()
abot = Agent(model, tools=[tool], checkpointer=memory,system=prompt)

In [21]:
]messages = [HumanMessage(content="What is the weather in Hyderabad and chennai?")

In [22]:
thread = {"configurable":{"thread_id":"1"}}

In [23]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_u8qdSfFe3GEkGUCJmkjpA9aL', 'function': {'arguments': '{"query": "weather in Hyderabad"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_x6LmHjYv7zfM7Ff3aoe3iNFN', 'function': {'arguments': '{"query": "weather in Chennai"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 156, 'total_tokens': 211}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-927d4d0e-c8d3-475d-9076-a6b0fa896cfe-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Hyderabad'}, 'id': 'call_u8qdSfFe3GEkGUCJmkjpA9aL'}, {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Chennai'}, 'id': 'call_x6LmHjYv7zfM7Ff3aoe3iNFN'}])]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Hyderabad'}, 'id': 'c

In [25]:
messages = [HumanMessage(content="Which has the highest temperature?")]
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content="Chennai has the highest temperature with 26°C compared to Hyderabad's 26°C.", response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 1959, 'total_tokens': 1978}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2196c902-6a08-4d35-87b9-03c9c5452e48-0')]


In [28]:
thread2 = {"configurable":{"thread_id":"4"}}
messages = [HumanMessage(content="So which is warmer ?")]
for event in abot.graph.stream({"messages": messages}, thread2):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rNPD6yGxx6XEM5DTfXtH8DA6', 'function': {'arguments': '{"query":"temperature in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 151, 'total_tokens': 172}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0ad297f4-8833-417c-b319-c48a3c7f9517-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'temperature in Los Angeles'}, 'id': 'call_rNPD6yGxx6XEM5DTfXtH8DA6'}])]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'temperature in Los Angeles'}, 'id': 'call_rNPD6yGxx6XEM5DTfXtH8DA6'}
Back to the model!
[ToolMessage(content='[{\'url\': \'https://www.accuweather.com/en/us/los-angeles/90012/current-weather/347625\', \'content\': \'Current weather in Los Angeles, CA. Check current conditions in Los Angeles, CA with r

In [29]:
thread2 = {"configurable":{"thread_id":"1"}}
messages = [HumanMessage(content="So which is warmer ?")]
for event in abot.graph.stream({"messages": messages}, thread2):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='Both Hyderabad and Chennai have the same temperature of 26°C, so they are equally warm at the moment.', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 1990, 'total_tokens': 2013}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c2dfb03f-79fe-4bba-a0aa-1cebf90a1adb-0')]
